In [9]:
import sys
from pathlib import Path
project_root = Path("..").resolve()
sys.path.insert(0, str(project_root))

import numpy as np
import time
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier as SklearnDT

from arboresque import DecisionTreeClassifier

iris = datasets.load_iris()
wine = datasets.load_wine()

The iris dataset contains 150 samples of sepal and petal measurement data, 4 features in all, for three types of irises, Setosa, Versicolor and Virginica.

In [10]:
for i in [12, 13, 86, 140, 75]:
    print(iris.data[i], iris.target[i])

[4.8 3.  1.4 0.1] 0
[4.3 3.  1.1 0.1] 0
[6.7 3.1 4.7 1.5] 1
[6.7 3.1 5.6 2.4] 2
[6.6 3.  4.4 1.4] 1


In [11]:
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

clf = DecisionTreeClassifier()  # default criterion="gini"
clf.fit(X_train, y_train)

print("Iris classifier")
print("Train accuracy:", clf.score(X_train, y_train))
print("Test accuracy:", clf.score(X_test, y_test))

Iris classifier
Train accuracy: 1.0
Test accuracy: 0.9111111111111111


In [12]:
y_pred = clf.predict(X_test)

print("Classification report:\n")
print(classification_report(y_test, y_pred))

print("Confusion matrix:\n")
print(confusion_matrix(y_test, y_pred))

Classification report:

              precision    recall  f1-score   support

           0       1.00      0.87      0.93        15
           1       0.79      1.00      0.88        15
           2       1.00      0.87      0.93        15

    accuracy                           0.91        45
   macro avg       0.93      0.91      0.91        45
weighted avg       0.93      0.91      0.91        45

Confusion matrix:

[[13  2  0]
 [ 0 15  0]
 [ 0  2 13]]


In [13]:
criterions = ["gini", "entropy"]

results = []
for crit in criterions:
    start = time.time()
    clf = DecisionTreeClassifier(criterion=crit)
    clf.fit(X_train, y_train)
    end = time.time()
    tm = end-start
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    print(f"Criterion: {crit}")
    print(f"    Train accuracy: {train_acc:.3f}")
    print(f"    Test accuracy:  {test_acc:.3f}")
    print(f"    Time: {tm:.3f}")
    print(f"    Depth: {clf.get_depth()}")
    print()

Criterion: gini
    Train accuracy: 1.000
    Test accuracy:  0.911
    Time: 0.026
    Depth: 5

Criterion: entropy
    Train accuracy: 1.000
    Test accuracy:  0.844
    Time: 0.010
    Depth: 8



In [14]:
clf.get_depth()

8

In [15]:
X_sample = X_test[:5]
y_sample = y_test[:5]

probs = clf.predict_proba(X_sample)
preds = clf.predict(X_sample)

print("Sample true labels: ", y_sample)
print("Predicted labels:   ", preds)
print("Predicted probs (rows):")
print(probs)
print("Row sums (should be 1):", probs.sum(axis=1))

Sample true labels:  [2 2 0 0 1]
Predicted labels:    [2 2 0 1 1]
Predicted probs (rows):
[[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
Row sums (should be 1): [1. 1. 1. 1. 1.]


In [17]:
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Test 1: max_features variations")
print()

for max_feat in [None, 2, 0.5, 'sqrt', 'log2']:
    clf = DecisionTreeClassifier(max_features=max_feat, random_state=42)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print(f"max_features={max_feat}: Accuracy={acc:.3f}, Depth={clf.get_depth()}, Leaves={clf.get_n_leaves()}")

print("Test 2: min_samples_split & min_samples_leaf")
print()

for min_split, min_leaf in [(2, 1), (10, 5), (20, 10), (0.1, 0.05)]:
    clf = DecisionTreeClassifier(min_samples_split=min_split, min_samples_leaf=min_leaf, random_state=42)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print(f"min_split={min_split}, min_leaf={min_leaf}: Acc={acc:.3f}, Depth={clf.get_depth()}, Leaves={clf.get_n_leaves()}")

print("Test 3: min_impurity_decrease")
print()

for min_imp in [0.0, 0.01, 0.05, 0.1]:
    clf = DecisionTreeClassifier(min_impurity_decrease=min_imp, random_state=42)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print(f"min_impurity_decrease={min_imp}: Acc={acc:.3f}, Depth={clf.get_depth()}, Leaves={clf.get_n_leaves()}")

print("Test 4: random_state reproducibility")
print()

clf1 = DecisionTreeClassifier(max_features=2, random_state=42)
clf1.fit(X_train, y_train)
pred1 = clf1.predict(X_test)

clf2 = DecisionTreeClassifier(max_features=2, random_state=42)
clf2.fit(X_train, y_train)
pred2 = clf2.predict(X_test)

print(f"Same random_state: Predictions identical? {np.array_equal(pred1, pred2)}")

clf3 = DecisionTreeClassifier(max_features=2, random_state=99)
clf3.fit(X_train, y_train)
pred3 = clf3.predict(X_test)

print(f"Different random_state: Predictions different? {not np.array_equal(pred1, pred3)}")

print("Test 5: Categorical features")
print()

# Create dataset with categorical feature
X_cat = np.column_stack([X, np.random.randint(0, 3, size=len(X))])  # Add categorical column
X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_cat, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(categorical_features=[4], random_state=42)
clf.fit(X_train_cat, y_train)
acc = clf.score(X_test_cat, y_test)
print(f"With categorical feature at index 4: Accuracy={acc:.3f}")
print(f"Features after encoding: {clf.n_features} (was {X_cat.shape[1]})")

print("Test 6: Comparison with sklearn")
print()

X, y = wine.data, wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

your_clf = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
your_clf.fit(X_train, y_train)
your_acc = your_clf.score(X_test, y_test)

sklearn_clf = SklearnDT(
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
sklearn_clf.fit(X_train, y_train)
sklearn_acc = sklearn_clf.score(X_test, y_test)

print(f"Your implementation: Acc={your_acc:.3f}, Depth={your_clf.get_depth()}, Leaves={your_clf.get_n_leaves()}")
print(f"Sklearn:           Acc={sklearn_acc:.3f}, Depth={sklearn_clf.get_depth()}, Leaves={sklearn_clf.get_n_leaves()}")
print(f"Accuracy difference: {abs(your_acc - sklearn_acc):.3f}")

print("Test 7: Edge cases")
print("=" * 50)

# Very restrictive parameters
clf = DecisionTreeClassifier(max_depth=1, min_samples_split=100, random_state=42)
clf.fit(X_train, y_train)
print(f"Very restrictive tree: Depth={clf.get_depth()}, Leaves={clf.get_n_leaves()}")

# Single sample per leaf
clf = DecisionTreeClassifier(min_samples_leaf=1, min_samples_split=2, random_state=42)
clf.fit(X_train, y_train)
print(f"Max flexibility tree: Depth={clf.get_depth()}, Leaves={clf.get_n_leaves()}")

print("Test 8: predict_proba")
print()

clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test[:5])
preds = clf.predict(X_test[:5])

print("First 5 samples:")
for i in range(5):
    print(f"  Predicted class: {preds[i]}, Probabilities: {proba[i]}")
    print(f"  Sum of probabilities: {proba[i].sum():.3f}")

print("All tests completed.")
print()


Test 1: max_features variations



KeyboardInterrupt: 

In [ ]:
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

clf = SklearnDT(random_state=0)
clf.fit(X_train, y_train)
print("Iris sklearn DecisionTreeClassifier (default gini)")
print("Train accuracy:", clf.score(X_train, y_train))
print("Test accuracy:", clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

from time import time

for crit in ["gini", "entropy"]:
    t0 = time()
    clf = SklearnDT(criterion=crit, random_state=0)
    clf.fit(X_train, y_train)
    t1 = time()
    print(f"\nCriterion: {crit}")
    print("  Train accuracy:", round(clf.score(X_train, y_train), 3))
    print("  Test accuracy: ", round(clf.score(X_test, y_test), 3))
    print("  Fit time (s):  ", round(t1 - t0, 6))
    print(f"  Depth: {clf.get_depth()}")


Iris sklearn DecisionTreeClassifier (default gini)
Train accuracy: 1.0
Test accuracy: 0.9777777777777777

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.94      1.00      0.97        15
           2       1.00      0.93      0.97        15

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

Confusion matrix:
 [[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]

Criterion: gini
  Train accuracy: 1.0
  Test accuracy:  0.978
  Fit time (s):   0.0
  Depth: 4

Criterion: entropy
  Train accuracy: 1.0
  Test accuracy:  0.956
  Fit time (s):   0.0
  Depth: 7
